<a href="https://colab.research.google.com/github/Gokkulnath/end/blob/main/Submission_END3_Session_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare the dataset 

## Download 
!wget http://www.cs.cmu.edu/~ark/QA-data/data/Question_Answer_Dataset_v1.2.tar.gz
!tar -xvf Question_Answer_Dataset_v1.2.tar.gz
!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv

--2021-12-02 17:18:32--  http://www.cs.cmu.edu/~ark/QA-data/data/Question_Answer_Dataset_v1.2.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8254496 (7.9M) [application/x-gzip]
Saving to: ‘Question_Answer_Dataset_v1.2.tar.gz.2’

Question_Answer_Dat 100%[===================>]   7.87M   531KB/s    in 22s     

2021-12-02 17:18:55 (370 KB/s) - ‘Question_Answer_Dataset_v1.2.tar.gz.2’ saved [8254496/8254496]

Question_Answer_Dataset_v1.2/
Question_Answer_Dataset_v1.2/S08/
Question_Answer_Dataset_v1.2/S08/question_answer_pairs.txt
Question_Answer_Dataset_v1.2/S08/data/
Question_Answer_Dataset_v1.2/S08/data/set4/
Question_Answer_Dataset_v1.2/S08/data/set4/a6.txt.clean
Question_Answer_Dataset_v1.2/S08/data/set4/a3.txt.clean
Question_Answer_Dataset_v1.2/S08/data/set4/a3.txt
Question_Answer_Dataset_v1.2/S08/data/set4/a5.txt
Question_Answer_Dataset_v1

In [ ]:
# Quora Duplicate Question : 
import pandas as pd
df = pd.read_csv('quora_duplicate_questions.tsv', sep='\t')
print(df.shape)
df.dropna(inplace=True)

(404290, 6)


In [ ]:
SOS_token = 0
EOS_token = 1

class Question_Lang:
  def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

  def addSentence(self, sentence):
      for word in sentence.split(' '):
          self.addWord(word)

  def addWord(self, word):
      if word not in self.word2index:
          self.word2index[word] = self.n_words
          self.word2count[word] = 1
          self.index2word[self.n_words] = word
          self.n_words += 1
      else:
          self.word2count[word] += 1
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    # print(s)
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


def readQuestions(ques1, ques2, reverse=False):
    df = pd.read_csv('quora_duplicate_questions.tsv', sep='\t')
    df.dropna(inplace=True)
    # # Read the file and split into lines
    # lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
    #     read().strip().split('\n')
        
    q1 = df['question1'].values
    q2 = df['question2'].values
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in q_arr] for q_arr in zip(q1,q2)]
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_q = Question_Lang(ques2)
        output_q = Question_Lang(ques1)
    else:
        input_q = Question_Lang(ques1)
        output_q = Question_Lang(ques2)

    return input_q, output_q, pairs




MAX_QUESTION_LENGTH = 50

# question_prefixes = (
#     "i am ", "i m ",
#     "he is", "he s ",
#     "she is", "she s ",
#     "you are", "you re ",
#     "we are", "we re ",
#     "they are", "they re "
# )


def filterPair(p):
    return len(p[0].split(' ')) < MAX_QUESTION_LENGTH and len(p[1].split(' ')) < MAX_QUESTION_LENGTH #\
        # and sp[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


def prepareData(q1, q2, reverse=False):
    input_q, output_q, pairs = readQuestions(q1, q2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_q.addSentence(pair[0])
        output_q.addSentence(pair[1])
    print("Counted words:")
    print(input_q.name, input_q.n_words)
    print(output_q.name, output_q.n_words)
    return input_q, output_q, pairs


input_q, output_q, pairs = prepareData('ques1', 'ques2', True)
print(random.choice(pairs))

Read 404287 sentence pairs
Trimmed to 337494 sentence pairs
Counting words...
Counted words:
ques2 49289
ques1 53975
['what are some of the best life changing books ?', 'daily life what are the best self improvement books ?']


In [ ]:
print(random.choice(pairs))

['how many w solar panels do i need to charge a battery of ah in hour ?', 'how many w solar panels do i need to charge a battery of ah in an hour ?']


In [ ]:
pairs[0:5]

[['what is the step by step guide to invest in share market ?',
  'what is the step by step guide to invest in share market in india ?'],
 ['what would happen if the indian government stole the kohinoor koh i noor diamond back ?',
  'what is the story of kohinoor koh i noor diamond ?'],
 ['how can internet speed be increased by hacking through dns ?',
  'how can i increase the speed of my internet connection while using a vpn ?'],
 ['find the remainder when math math is divided by ?',
  'why am i mentally very lonely ? how can i solve it ?'],
 ['which fish would survive in salt water ?',
  'which one dissolve in water quikly sugar salt methane and carbon di oxide ?']]

In [ ]:
input_q

In [ ]:
help(input_q)

Help on Question_Lang in module __main__ object:

class Question_Lang(builtins.object)
 |  Question_Lang(name)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  addSentence(self, sentence)
 |  
 |  addWord(self, word)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [ ]:
input_q.__dict__.items()

dict_items([('name', 'ques2'), ('word2index', {'what': 2, 'is': 3, 'the': 4, 'step': 5, 'by': 6, 'guide': 7, 'to': 8, 'invest': 9, 'in': 10, 'share': 11, 'market': 12, '?': 13, 'would': 14, 'happen': 15, 'if': 16, 'indian': 17, 'government': 18, 'stole': 19, 'kohinoor': 20, 'koh': 21, 'i': 22, 'noor': 23, 'diamond': 24, 'back': 25, 'how': 26, 'can': 27, 'internet': 28, 'speed': 29, 'be': 30, 'increased': 31, 'hacking': 32, 'through': 33, 'dns': 34, 'find': 35, 'remainder': 36, 'when': 37, 'math': 38, 'divided': 39, 'which': 40, 'fish': 41, 'survive': 42, 'salt': 43, 'water': 44, 'keeps': 45, 'childern': 46, 'active': 47, 'and': 48, 'far': 49, 'from': 50, 'phone': 51, 'video': 52, 'games': 53, 'should': 54, 'do': 55, 'a': 56, 'great': 57, 'geologist': 58, 'you': 59, 'use': 60, 'instead': 61, 'of': 62, 'hack': 63, 'motorola': 64, 'dcx': 65, 'for': 66, 'free': 67, 'see': 68, 'all': 69, 'my': 70, 'youtube': 71, 'comments': 72, 'make': 73, 'physics': 74, 'easy': 75, 'learn': 76, 'was': 77, 

In [ ]:
input_size = input_q.n_words
hidden_size = 128
input_size

49289

In [ ]:
embedding = nn.Embedding(input_size, hidden_size).to(device)
gru = nn.GRU(hidden_size, hidden_size).to(device)

In [ ]:
sample = random.choice(pairs)
sample,device

(['can i prepare for cat without any coaching ? .i am the student of bpt st year',
  'i am in st year of engineering how should i start my preparation for cat ?'],
 device(type='cuda'))

In [ ]:
# Convert String --> indices (list) + Add EOS and SOS --> Tensor
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_q, pair[0])
    target_tensor = tensorFromSentence(output_q, pair[1])
    return (input_tensor, target_tensor)


In [ ]:
import torch.nn as nn
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_QUESTION_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output, hidden = self.gru(output, hidden)
        output = F.relu(output)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_QUESTION_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))



In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_QUESTION_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_q, sentence)
        input_length = input_tensor.size()[0]
        print(input_length)
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 128
encoder1 = EncoderRNN(input_q.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_q.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

6m 16s (- 87m 51s) (5000 6%) 5.1036


In [ ]:
%pdb